<a href="https://colab.research.google.com/github/RogSwoyam45/Artificial-Neural-network/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install kaggle

In [6]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.38GB/s]


In [8]:
from zipfile import ZipFile
file_name = "/content/sentiment140.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()


In [9]:
import numpy as np
import pandas as pd

In [10]:
twitter_data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv" , encoding='ISO-8859-1')

In [11]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [12]:
twitter_data.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']
twitter_data = twitter_data[['sentiment', 'text']]

In [13]:
twitter_data.head()

,sentiment,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [14]:
data = twitter_data['text']

In [15]:
data

,text
0,is upset that he can't update his Facebook by ...
1,@Kenichan I dived many times for the ball. Man...
2,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all...."
4,@Kwesidei not the whole crew
...,...
1599994,Just woke up. Having no school is the best fee...
1599995,TheWDB.com - Very cool to hear old Walt interv...
1599996,Are you ready for your MoJo Makeover? Ask me f...
1599997,Happy 38th Birthday to my boo of alll time!!! ...


In [16]:
import re

In [17]:
data = data.apply(lambda x: re.sub(r'@\w+', '', x))

In [18]:
data[3]

" no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. "

In [19]:
data = data[:100]
data.shape

(100,)

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [21]:
tokenizer = Tokenizer()

tokenizer.fit_on_texts(data)

In [22]:
len(tokenizer.word_index)

657

In [23]:
input_sequences = []
for sentence in data:
  tokenized_sentence = tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenizer.word_index)):
    input_sequences.append(tokenized_sentence[:i+1])

In [24]:
input_sequences

[[8, 183],
 [8, 183, 11],
 [8, 183, 11, 184],
 [8, 183, 11, 184, 52],
 [8, 183, 11, 184, 52, 185],
 [8, 183, 11, 184, 52, 185, 186],
 [8, 183, 11, 184, 52, 185, 186, 104],
 [8, 183, 11, 184, 52, 185, 186, 104, 105],
 [8, 183, 11, 184, 52, 185, 186, 104, 105, 187],
 [8, 183, 11, 184, 52, 185, 186, 104, 105, 187, 9],
 [8, 183, 11, 184, 52, 185, 186, 104, 105, 187, 9, 4],
 [8, 183, 11, 184, 52, 185, 186, 104, 105, 187, 9, 4, 188],
 [8, 183, 11, 184, 52, 185, 186, 104, 105, 187, 9, 4, 188, 72],
 [8, 183, 11, 184, 52, 185, 186, 104, 105, 187, 9, 4, 188, 72, 53],
 [8, 183, 11, 184, 52, 185, 186, 104, 105, 187, 9, 4, 188, 72, 53, 6],
 [8, 183, 11, 184, 52, 185, 186, 104, 105, 187, 9, 4, 188, 72, 53, 6, 189],
 [8,
  183,
  11,
  184,
  52,
  185,
  186,
  104,
  105,
  187,
  9,
  4,
  188,
  72,
  53,
  6,
  189,
  106],
 [8,
  183,
  11,
  184,
  52,
  185,
  186,
  104,
  105,
  187,
  9,
  4,
  188,
  72,
  53,
  6,
  189,
  106,
  43],
 [8,
  183,
  11,
  184,
  52,
  185,
  186,
  104,
 

In [25]:
max_len = max([len(x) for x in input_sequences])

In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences = pad_sequences(input_sequences, maxlen = max_len, padding='pre')

In [27]:
padded_input_sequences

array([[  0,   0,   0, ...,   0,   8, 183],
       [  0,   0,   0, ...,   8, 183,  11],
       [  0,   0,   0, ..., 183,  11, 184],
       ...,
       [  0,   0,   0, ..., 656,  89, 657],
       [  0,   0,   0, ..., 656,  89, 657],
       [  0,   0,   0, ..., 656,  89, 657]], dtype=int32)

In [28]:
X = padded_input_sequences[:,:-1]
y = padded_input_sequences[:,-1]

In [29]:
X.shape


(65600, 27)

In [30]:
y.shape

(65600,)

In [31]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=658)

In [32]:
y.shape

(65600, 658)

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



model = Sequential()
model.add(Embedding(658, 100, input_length=56))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(150))
model.add(Dense(658, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [34]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [37]:
model.fit(X,y,epochs=10)

Epoch 1/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9878 - loss: 0.0517
Epoch 2/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9903 - loss: 0.0393
Epoch 3/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9928 - loss: 0.0305
Epoch 4/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9940 - loss: 0.0253
Epoch 5/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9955 - loss: 0.0192
Epoch 6/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9967 - loss: 0.0146
Epoch 7/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - accuracy: 0.9978 - loss: 0.0107
Epoch 8/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9983 - loss: 0.0086
Epoch 9/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9984 - loss: 0.0074
Epoch 10/10
2050/2050 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9990 - loss: 0.0053


In [41]:
text = "is"

for i in range(10):
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=56, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      text = text + " " + word
      print(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
is upset
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
is upset that
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
is upset that he
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
is upset that he can't
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
is upset that he can't update
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
is upset that he can't update his
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
is upset that he can't update his facebook
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
is upset that he can't update his facebook by
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
is upset that he can't update his facebook by texting
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
is upset that he can't update his facebook by texting it
